# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Print scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.20.2


In [2]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean
import time
import datetime

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('messages_categories',engine)
X = df['message']
Y = df.iloc[:,3:-2]

In [4]:
# Print category columns
category_cols = df.columns[3:-2].tolist()
print(category_cols)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [5]:
# Get stop words in 'English' language
stop_words = stopwords.words("english")

In [6]:
# Print length of stop words in English language
print('Length of stop words in English language is {}'.format(len(stop_words)))

Length of stop words in English language is 179


In [7]:
# Print stop words in English language
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# Check if any message contain URL link
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url_count = pd.Series([])
url_count = X.apply(lambda message: len(re.findall(url_regex, message)))
print(type(url_count))
url_count.value_counts().sort_index()    

<class 'pandas.core.series.Series'>


0    25364
1      559
2       78
3       23
4        2
5        1
6        1
Name: message, dtype: int64

From above, we can observe that most of the messages does not have URL links but there are few messages which do contain URL links. There is only 1 observation which contains 5 URL links.

In [9]:
# Define function tokenize to normalize, tokenize and lemmatize text string
def tokenize(text):
    """Normalize, tokenize and lemmatize text string
    
    Args:
    text: string, String containing message for processing
       
    Returns:
    clean_tokens: list, List containing normalized and lemmatized word tokens
    """
    
    # Replace URL links in text string with string 'urlplaceholder'
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Substitute characters in text string which match regular expression r'[^a-zA-Z0-9]'
    # with single whitespace
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Get word tokens from text string
    tokens = word_tokenize(text)
    
    # Instantiate WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Clean tokens
    clean_tokens = []
    for tok in tokens:
        # convert token to lowercase as stop words are in lowercase
        tok_low = tok.lower() 
        if tok_low not in stop_words:
            # Lemmatize token and remove the leading and trailing spaces from lemmatized token
            clean_tok = lemmatizer.lemmatize(tok_low).lower().strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [10]:
# Print first 5 messages and their respective tokens
for idx in X.index.tolist()[0:5]:
    print(X.loc[idx])
    print('-'*100)
    print(tokenize(X.loc[idx]))
    print('*'*100)

Weather update - a cold front from Cuba that could pass over Haiti
----------------------------------------------------------------------------------------------------
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
****************************************************************************************************
Is the Hurricane over or is it not over
----------------------------------------------------------------------------------------------------
['hurricane']
****************************************************************************************************
Looking for someone but no name
----------------------------------------------------------------------------------------------------
['looking', 'someone', 'name']
****************************************************************************************************
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
-------------------------------------

In [11]:
# Print message with url count of 5
print('Index of message with 5 URL links is {}'.format(url_count[url_count == 5].index[0]))
X[url_count[url_count == 5].index[0]]

Index of message with 5 URL links is 12598


'Hurricane Sandy Flight Cancellations: Thousands Of Flights Canceled Due. http://t.co/DMo0tbQE Most read by neighbors in #Roseville #Newarkhappy halloween 2012 (@Frankenstorm Apocalypse - Hurricane Sandy w/ 213 others) http://t.co/DTw9W3kKThe protective cover for the Enterprise failed last night. #NYC #sandy @Space Shuttle Enterprise http://t.co/5jexF6ZG@StormTeam8 @CTPostTrumbull Shelbourne rd Trumbull right next door. Never lost power http://t.co/NqMyZ1H8RT @nytimes: The New York Times is providing free, unlimited access to storm coverage on http://t.co/HkHYUWhW and its mobile apps today.'

In [12]:
# Print message with index location 12598 and its respective tokens
# Index location 12598 contain message with 5 URL links
print(X.loc[12598])
print('-'*100)
print(tokenize(X.loc[12598]))

Hurricane Sandy Flight Cancellations: Thousands Of Flights Canceled Due. http://t.co/DMo0tbQE Most read by neighbors in #Roseville #Newarkhappy halloween 2012 (@Frankenstorm Apocalypse - Hurricane Sandy w/ 213 others) http://t.co/DTw9W3kKThe protective cover for the Enterprise failed last night. #NYC #sandy @Space Shuttle Enterprise http://t.co/5jexF6ZG@StormTeam8 @CTPostTrumbull Shelbourne rd Trumbull right next door. Never lost power http://t.co/NqMyZ1H8RT @nytimes: The New York Times is providing free, unlimited access to storm coverage on http://t.co/HkHYUWhW and its mobile apps today.
----------------------------------------------------------------------------------------------------
['hurricane', 'sandy', 'flight', 'cancellation', 'thousand', 'flight', 'canceled', 'due', 'urlplaceholder', 'read', 'neighbor', 'roseville', 'newarkhappy', 'halloween', '2012', 'frankenstorm', 'apocalypse', 'hurricane', 'sandy', 'w', '213', 'others', 'urlplaceholder', 'protective', 'cover', 'enterpris

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
# Create a basic pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
#start_time = time.time()
start_datetime = datetime.datetime.now().replace(microsecond=0)

# Train basic pipeline
pipeline.fit(X_train, Y_train)
#print("--- %s seconds ---" % (time.time() - start_time))
print("--- Training time: %s ---" % (datetime.datetime.now().replace(microsecond=0) - start_datetime))

--- Training time: 0:00:58 ---


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
# Predict categories from test set
start_datetime = datetime.datetime.now().replace(microsecond=0)
Y_pred = pipeline.predict(X_test)
print("--- Predicting time: %s ---" % (datetime.datetime.now().replace(microsecond=0) - start_datetime))

--- Predicting time: 0:00:05 ---


In [16]:
# Print type and shape of Y_test and Y_pred
print('Y_test has type: {} and its shape is: {}'.format(type(Y_test), Y_test.shape))
print('Y_pred has type: {} and its shape is: {}'.format(type(Y_pred), Y_pred.shape))

Y_test has type: <class 'pandas.core.frame.DataFrame'> and its shape is: (6507, 35)
Y_pred has type: <class 'numpy.ndarray'> and its shape is: (6507, 35)


In [17]:
# Print first 5 rows of Y_test dataframe
Y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,1,0,0,1,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
25482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13741,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
14548,1,0,0,1,1,1,1,0,1,0,...,0,1,1,1,1,1,1,1,1,0


In [18]:
# Print first 5 rows in Y_pred ndarray
Y_pred[0:5]

array([[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int64)

In [19]:
# Print accuracy of basic pipeline for each of individual category 
accuracy = (Y_pred == Y_test).mean()
accuracy

related                   0.806977
request                   0.883049
offer                     0.996004
aid_related               0.747195
medical_help              0.919010
medical_products          0.951130
search_and_rescue         0.970340
security                  0.979560
military                  0.966190
water                     0.951591
food                      0.929614
shelter                   0.933917
clothing                  0.986169
money                     0.974950
missing_people            0.989550
refugees                  0.969725
death                     0.961426
other_aid                 0.871830
infrastructure_related    0.934378
transport                 0.957277
buildings                 0.951437
electricity               0.980944
tools                     0.993853
hospitals                 0.989089
shops                     0.994621
aid_centers               0.988935
other_infrastructure      0.956047
weather_related           0.862609
floods              

In [20]:
# Print overall accuracy of basic pipeline
overall_accuracy = (Y_pred == Y_test).mean().mean()
print('Overall accuracy of basic pipeline is: {}%'.format(round(overall_accuracy*100, 2)))

Overall accuracy of basic pipeline is: 94.29%


In [23]:
# Define function to calculate the multi-label f-score
def multi_label_fscore(y_true, y_pred, beta=1):
    """Calculate individual weighted average fbeta score of each category and
    geometric mean of weighted average fbeta score of each category
    
    Args:
    y_true: dataframe, dataframe containing true labels i.e. Y_test
    y_pred: ndarray, ndarray containing predicted labels i.e. Y_pred
    beta: numeric, beta value
       
    Returns:
    f_score_gmean: float, geometric mean of fbeta score for each category
    """
    
    b = beta
    f_score_dict = {}
    score_list = []
    
    # Create dataframe y_pred_df from ndarray y_pred 
    y_pred_df = pd.DataFrame(y_pred, columns=y_true.columns)

    for column in y_true.columns:
        score = round(fbeta_score(y_true[column], y_pred_df[column], beta, average='weighted'),4)
        score_list.append(score)
    f_score_dict['category'] = y_true.columns.tolist()
    f_score_dict['f_score'] = score_list

   
    f_score_df = pd.DataFrame.from_dict(f_score_dict)

    f_score_gmean = gmean(f_score_df['f_score'])

    return f_score_gmean

In [24]:
# Print overall f_score of basic pipeline
multi_f_gmean = multi_label_fscore(Y_test,Y_pred, beta = 1)
print('Overall F_beta_score is: {0:.2f}%'.format(multi_f_gmean*100))

Overall F_beta_score is: 92.84%


C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
# Report the f1 score, precision and recall for each output category of the dataset by iterating through the columns
# and calling sklearn's classification_report on each column
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('CATEGORY: {}\n'.format(column))
    print(classification_report(Y_test[column],pd.DataFrame(Y_pred, columns=Y_test.columns)[column]))

C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


------------------------------------------------------

CATEGORY: related

              precision    recall  f1-score   support

           0       0.64      0.46      0.54      1569
           1       0.84      0.92      0.88      4938

   micro avg       0.81      0.81      0.81      6507
   macro avg       0.74      0.69      0.71      6507
weighted avg       0.79      0.81      0.80      6507

------------------------------------------------------

CATEGORY: request

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5403
           1       0.78      0.43      0.56      1104

   micro avg       0.88      0.88      0.88      6507
   macro avg       0.84      0.70      0.74      6507
weighted avg       0.87      0.88      0.87      6507

------------------------------------------------------

CATEGORY: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1  


------------------------------------------------------

CATEGORY: electricity

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6380
           1       0.67      0.05      0.09       127

   micro avg       0.98      0.98      0.98      6507
   macro avg       0.82      0.52      0.54      6507
weighted avg       0.98      0.98      0.97      6507

------------------------------------------------------

CATEGORY: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6467
           1       0.00      0.00      0.00        40

   micro avg       0.99      0.99      0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

------------------------------------------------------

CATEGORY: hospitals

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6436
       

### 6. Improve your model
Use grid search to find better parameters. 

#### 6.1 Add custom Estimator

In [33]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
#        print('*'*100)
#        print(sentence_list)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
#            print(pos_tags)
#            print(type(pos_tags))
            if pos_tags:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return float(True)
        return float(False)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [34]:
type(X_train[:5])

pandas.core.series.Series

In [35]:
type(pd.Series(X_train[:5]))

pandas.core.series.Series

In [36]:
for idx in X_train.index[0:5]:
    print(idx, X_train[idx])

2796 Petit Goave #1 needs food. Where can we sleep. Please, we're asking not take years because we can't survive?
16642 Monsoon downpours and upstream waters cascading down the hills from across the border in northeastern India, swamped towns and villages in Bangladesh inundating mud walled dwellings, rice farms, roads, bridges and factories.
17486 Cyclone Nargis hit the densely populated delta last month, killing up to 134,000 people and leaving 2.4 million destitute.
4034 I received all your SMS but I never got any aid where I live. Thank you. 
25729 In the South-South states of Delta and Rivers, VAWG incidents often occur within the context of domestic, criminal and cult violence; with women being both targets as well as innocent bystanders swept up in the general insecurity.


In [37]:
SVE = StartingVerbExtractor()
#SVE.fit(X_train[:5]).tranform(X_train)
dir(SVE)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_param_names',
 'fit',
 'fit_transform',
 'get_params',
 'set_params',
 'starting_verb',
 'transform']

In [38]:
SVE = StartingVerbExtractor()
SVE.starting_verb("Petit Goave #1 needs food. Where can we sleep. Please, we're asking not take years because we can't survive?")

1.0

#### 6.2 Improve pipeline

In [39]:
# Create a new improved pipeline
pipeline_new = Pipeline([('features', FeatureUnion([
                                        ('text_pipeline', Pipeline([
                                            ('vect', CountVectorizer(tokenizer=tokenize)),
                                            ('tfidf', TfidfTransformer())])),
                                        ('starting_verb', StartingVerbExtractor())])),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

#### 6.3 Specify parameters for grid search

In [40]:
# Specify parameters for grid search
parameters = {
    'features__text_pipeline__vect__ngram_range': [(1,2)],
    'features__text_pipeline__vect__max_df': [0.75],
    'features__text_pipeline__vect__max_features': [5000],
    'features__text_pipeline__tfidf__use_idf': [True],
#   'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#   'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#   'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#   'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [200],
    'clf__estimator__min_samples_split': [4],
#    'clf__estimator__n_estimators': [50,100,200],
#    'clf__estimator__min_samples_split': [2,3,4],
    'features__transformer_weights': (
        {'text_pipeline': 1, 'starting_verb': 0.5},
#        {'text_pipeline': 0.5, 'starting_verb': 1},
#        {'text_pipeline': 0.8, 'starting_verb': 1},
    )
}

#### 6.3 Define custom scorer

In [41]:
# Specify custom scorer
scorer = make_scorer(multi_label_fscore,greater_is_better = True)

#### 6.4 Create grid search object

In [42]:
# create grid search object
cv = GridSearchCV(pipeline_new, param_grid=parameters, scoring=scorer,verbose = 2)

In [101]:
# Print grid search CV object params
from pprint import pprint
import json

#data = json.dumps(my_dict, indent=1)
#data=pipeline_new.get_params().keys()
pprint(pipeline_new.get_params())
#type(pipeline_new.get_params())

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__clas

In [44]:
# Fit GridSearchCV object to training set
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, total= 6.3min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.7min remaining:    0.0s


C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, total= 6.3min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, total= 6.7min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 24.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'features__text_pipeline__vect__ngram_range': [(1, 2)], 'features__text_pipeline__vect__max_df': [0.75], 'features__text_pipeline__vect__max_features': [5000], 'features__text_pipeline__tfidf__use_idf': [True], 'clf__estimator__n_estimators': [200], 'clf__estimator__min_samples_split': [4], 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5},)},
       pre_dispatch='2*n_jobs', refit=True, return_t

**After trying various combinations of parameters defined in parameter's grid, we have the best estimator as per the metrics on the training set and below are the details for best estimator and parameters.**

In [103]:
# Print best estimator for GridSearchCV object
cv.best_estimator_

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [110]:
# Print best estimator's all parameters
pprint(cv.best_estimator_.get_params())

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_weig

In [108]:
# Print the score(as per custom scorer) on the training set after the best estimator selected has been refit 
cv.best_score_

0.9362539803390713

In [111]:
# Print best estimator parameters selected from the parameter's grid with their values 
cv.best_params_

{'clf__estimator__min_samples_split': 4,
 'clf__estimator__n_estimators': 200,
 'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_df': 0.75,
 'features__text_pipeline__vect__max_features': 5000,
 'features__text_pipeline__vect__ngram_range': (1, 2),
 'features__transformer_weights': {'text_pipeline': 1, 'starting_verb': 0.5}}

In [113]:
# Print scorer used to select the best parameters for the GridSearchCV object
cv.scorer_

make_scorer(multi_label_fscore)

In [115]:
# Print the number of cross-validation splits (folds/iterations) used while fitting the training set
cv.n_splits_

3

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.